In [1]:
#import

import pandas as pd
from sklearn import linear_model

In [2]:
#서울 동대문구 인구밀도 정제화

df_people_dd = pd.read_csv("dongdaemun_people.csv", skiprows=1, encoding="utf-8")
df_people_dd.drop(columns='동별(1)', axis=0, inplace=True)
df_people_dd.columns = ['지역', '인구 (명)', '면적 (㎢)', '인구밀도 (명/㎢)']
df_people_dd.loc[0] = ('서울동대문구', 359873, 14.22, 25315.00)

In [3]:
#충남 당진시 인구밀도 정제화

df_people_dj = pd.read_csv("dangjin_people.csv", skiprows=1, encoding="utf-8")
df_people_dj.columns = ['지역', '인구 (명)', '인구밀도 (명/㎢)', '면적 (㎢)']
df_people_dj.loc[0] = ('충남당진시', 174079, 238.48, 705.53)

In [4]:
#동대문구, 당진시 인구밀도 merge

df_people_merged = pd.concat([df_people_dd, df_people_dj], ignore_index=True)
df_people_merged = df_people_merged.reset_index(drop=True)

In [5]:
df_people_merged

,지역,인구 (명),면적 (㎢),인구밀도 (명/㎢)
0,서울동대문구,359873,14.22,25315.00
1,충남당진시,174079,705.53,238.48


In [6]:
#동대문구, 당진시 범죄건수 정제화

df_crime = pd.read_csv("crime2023.csv", encoding='cp949')
df_crime = df_crime[['범죄중분류', '서울동대문구', '충남당진시']]

df_crime=df_crime.transpose()
df_crime = df_crime.rename(columns=df_crime.iloc[0])
df_crime = df_crime.drop(df_crime.index[0])

df_crime = df_crime[['살인기수', '강도', '강간', '절도범죄', '폭행']]
df_crime = df_crime.reset_index()

df_crime.columns = ['지역', '살인기수', '강도', '강간', '절도범죄', '폭행']

df_crime_all = pd.DataFrame({'지역': ['서울동대문구', '충남당진시'],
              '총범죄': [2159, 804]})

df_crime = pd.merge(df_crime, df_crime_all, on='지역', how='inner')

In [7]:
df_crime

,지역,살인기수,강도,강간,절도범죄,폭행,총범죄
0,서울동대문구,2,4,36,1278,839,2159
1,충남당진시,2,1,18,445,338,804


In [8]:
#인구밀도, 범죄건수 merge

df_merged = pd.merge(df_people_merged, df_crime, on='지역', how='inner')

In [9]:
df_merged

,지역,인구 (명),면적 (㎢),인구밀도 (명/㎢),살인기수,강도,강간,절도범죄,폭행,총범죄
0,서울동대문구,359873,14.22,25315.00,2,4,36,1278,839,2159
1,충남당진시,174079,705.53,238.48,2,1,18,445,338,804


In [10]:
#동대문 가로등 수 추출

df_dd_lights = pd.read_csv("dongdaemun.csv",  encoding = 'cp949')
df_dd_lights = df_dd_lights.fillna(0)
dd_lights = df_dd_lights['보행등'].sum()

In [11]:
#당진시 가로등 수 추출출

df_dj_lights = pd.read_csv("dangjin.csv")
df_dj_lights.columns = df_dj_lights.columns.str.strip()
df_dj_lights = df_dj_lights.fillna(0)
dj_lights = df_dj_lights['가로등(공원등미포)'].sum()

In [12]:
print(dd_lights)
print(dj_lights)

2939.0
6836.0


In [13]:
df_lights = pd.DataFrame({'지역': ['서울동대문구', '충남당진시'],
                          '가로등수': [2939.0, 6836.0]})
df_lights

,지역,가로등수
0,서울동대문구,2939.0
1,충남당진시,6836.0


In [14]:
#당진시 경찰시설 수 출처https://stat.kosis.kr/nsieu/view/tree.do?task=branchView&hOrg=705&id=705_001*MT_OTITLE
#동대문구 경찰시설 수 출처 통계연보 https://www.ddm.go.kr/ddmmap/contents.do?key=1653
#경찰관서 비율(개/㎢)

df_police = pd.DataFrame({'지역': ['서울동대문구', '충남당진시'],
                          '경찰관서 (개/㎢)': [21/14.22, 15/705.53]})

df_police

,지역,경찰관서 (개/㎢)
0,서울동대문구,1.476793
1,충남당진시,0.021261


In [15]:
#merge 및 실수형으로 변경

df_merged = pd.merge(df_merged, df_police, on='지역', how='inner')
df_merged = pd.merge(df_merged, df_lights, on= '지역', how= 'inner')

for col in ['인구 (명)', '면적 (㎢)', '인구밀도 (명/㎢)', '살인기수', '강도', '강간', '절도범죄', '폭행', '총범죄', '경찰관서 (개/㎢)', '가로등수']:
    df_merged[col] = df_merged[col].astype(float)

df_merged

,지역,인구 (명),면적 (㎢),인구밀도 (명/㎢),살인기수,강도,강간,절도범죄,폭행,총범죄,경찰관서 (개/㎢),가로등수
0,서울동대문구,359873.0,14.22,25315.00,2.0,4.0,36.0,1278.0,839.0,2159.0,1.476793,2939.0
1,충남당진시,174079.0,705.53,238.48,2.0,1.0,18.0,445.0,338.0,804.0,0.021261,6836.0


In [16]:
#종속변수: 총범죄건수

# 독립 변수와 종속 변수 정의

x_data = df_merged[['인구밀도 (명/㎢)', '가로등수', '경찰관서 (개/㎢)']]
y_all_crime = df_merged['총범죄']

# 다중 선형 회귀 모델 생성 및 학습

reg_all_crime = linear_model.LinearRegression()
reg_all_crime.fit(x_data, y_all_crime)

print('절편: ', reg_all_crime.intercept_)
print('회귀계수: ', reg_all_crime.coef_)

절편:  847.4674133955883
회귀계수:  [ 5.27604197e-02 -8.19919812e-03  3.06240710e-06]


In [17]:
#범죄 종류 별

# 독립 변수 정의

y_muder = df_merged['살인기수']
y_robbery = df_merged['강도']
y_rape = df_merged['강간']
y_thief = df_merged['절도범죄']
y_battery = df_merged['폭행']

# 다중 선형 회귀 모델 생성 및 학습

reg_muder = linear_model.LinearRegression()
reg_muder.fit(x_data, y_muder)

reg_robbery = linear_model.LinearRegression()
reg_robbery.fit(x_data, y_robbery)

reg_rape = linear_model.LinearRegression()
reg_rape.fit(x_data, y_rape)

reg_thief = linear_model.LinearRegression()
reg_thief.fit(x_data, y_thief)

reg_battery = linear_model.LinearRegression()
reg_battery.fit(x_data, y_battery)

print('절편: ')

print('-살인', reg_muder.intercept_)
print('-강도', reg_robbery.intercept_)
print('-강간', reg_rape.intercept_)
print('-절도', reg_thief.intercept_)
print('-폭행', reg_battery.intercept_)

print('회귀계수: ', reg_all_crime.coef_)

print('-살인', reg_muder.coef_)
print('-강도', reg_robbery.coef_)
print('-강간', reg_rape.coef_)
print('-절도', reg_thief.coef_)
print('-폭행', reg_battery.coef_)

절편: 
-살인 2.0
-강도 1.0962378156359887
-강간 18.57742689381593
-절도 471.72203347492604
-폭행 354.0717152112101
회귀계수:  [ 5.27604197e-02 -8.19919812e-03  3.06240710e-06]
-살인 [0. 0. 0.]
-강도 [ 1.16812737e-04 -1.81532062e-05  6.78023712e-09]
-강간 [ 7.00876424e-04 -1.08919237e-04  4.06814227e-08]
-절도 [ 3.24350034e-02 -5.04054025e-03  1.88264584e-06]
-폭행 [ 1.95077271e-02 -3.03158543e-03  1.13229960e-06]


In [18]:
#결과 정리

df = pd.DataFrame({'살인': reg_muder.coef_,
                   '강도': reg_robbery.coef_,
                   '강간': reg_rape.coef_,
                   '절도': reg_thief.coef_,
                   '폭행': reg_battery.coef_,
                   '총범죄': reg_all_crime.coef_})

df_df = pd.DataFrame({'살인': [reg_muder.intercept_],
                   '강도': [reg_robbery.intercept_],
                   '강간': [reg_rape.intercept_],
                   '절도': [reg_thief.intercept_],
                   '폭행': [reg_battery.intercept_],
                   '총범죄': [reg_all_crime.intercept_]})

df = pd.concat([df_df, df])

df = df.transpose()
df.columns = ['절편', '인구밀도 (명/㎢)', '가로등수', '경찰관서 (개/㎢)']


#다만 경찰관서에 관해서는 인과관계가 반대일 수 있음
df

,절편,인구밀도 (명/㎢),가로등수,경찰관서 (개/㎢)
살인,2.000000,0.000000,0.000000,0.000000e+00
강도,1.096238,0.000117,-0.000018,6.780237e-09
강간,18.577427,0.000701,-0.000109,4.068142e-08
절도,471.722033,0.032435,-0.005041,1.882646e-06
폭행,354.071715,0.019508,-0.003032,1.132300e-06
총범죄,847.467413,0.052760,-0.008199,3.062407e-06


전체적으로 가로등 수와 범죄 발생율은 반비례하는 경향이 있으며 경찰관서에 경우 비례하나 범죄 다수 발생 지역에 집중적으로 세워지는 등, 인과관계가 반대일 경우를 고려해보아야 할 것 같다.

살인의 경우, 건수가 2개에 동대문구, 당진시 두 곳에서 같은 값이기에 적절한 결과를 낼 수 없었다.